# Cloud Computing Assignment 2022-2023
Implementation of an application processing large data sets in parallel on a distributed Cloud environment (ie. AWS)

### Solution setup - Pre-requisites:
1. Make sure the aws credentials taken from the Learner Lab are updated in the ~/.aws/credentials file (Test connection locally using aws sts get-caller-identity)
2. Specify the "labsuser.pem" perm-key's (taken from the Learner Lab) path, needed by paramiko to connect to the EC2 instances and execute ssh commands.
3. Create EC2, S3 and SQS resources and clients using boto3.
### Solution setup steps (Using Boto3):
1. Create a cluster of EC2 instances on AWS, using the AWS Linux 2 images.
2. Create a S3 bucket to store the data.
3. Create a SQS queue to store stacks of messages.

### IMPORTS:

In [ ]:
import boto3
import numpy as np
import time
import paramiko

### CONFIGURATION:

In [ ]:
# Permission key:
pem_key = 'learner-lab-cfg/labsuser.pem'
# Create an EC2 resource (higher level abstraction than a client):
ec2 = boto3.resource('ec2')
ec2_client = boto3.client('ec2')
# Create a S3 resource:
s3 = boto3.resource('s3')
s3_client = boto3.client('s3')
# Create a SQS resource:
sqs = boto3.resource('sqs')
sqs_client = boto3.client('sqs')

### BOTO 3 - APP INTERFACE

In [ ]:
# Get boto3 session credentials, successfully authenticated using updated local credentials:
def get_boto3_session_credentials():
    session = boto3.session.Session()
    credentials = session.get_credentials()
    return credentials

### SQS - APP INTERFACE

In [ ]:
# CREATE SQS QUEUE:
def create_sqs_queue(queue_name):
    try:
        queue = sqs.create_queue(
            QueueName=queue_name,
            Attributes={
                'FifoQueue': 'true',
                'MessageRetentionPeriod': '86400'
            }
        )
        return queue
    except Exception as e:
        print(e)
        return None

# DELETE SQS QUEUE:
def delete_sqs_queue(queue_name):
    try:
        queue = sqs.get_queue_by_name(QueueName=queue_name)
        queue.delete()
        return True
    except Exception as e:
        print(e)
        return False

# SEND MESSAGE TO SQS QUEUE:
def send_message_to_sqs_queue(queue_name, message_body, message_group_id, message_deduplication_id):
    try:
        queue = sqs.get_queue_by_name(QueueName=queue_name)
        response = queue.send_message(
            MessageBody=message_body,
            MessageGroupId=message_group_id,
            MessageDeduplicationId=message_deduplication_id
        )
        return response
    except Exception as e:
        print(e)
        return None

# GET ALL MESSAGES FROM SQS QUEUE:
def get_all_messages_from_sqs_queue(queue_name):
    try:
        queue = sqs.get_queue_by_name(QueueName=queue_name)
        messages = queue.receive_messages()
        return messages
    except Exception as e:
        print(e)
        return None

# DELETE ALL MESSAGES FROM SQS QUEUE:
def delete_all_messages_from_sqs_queue(queue_name):
    try:
        queue = sqs.get_queue_by_name(QueueName=queue_name)
        messages = queue.receive_messages()
        for message in messages:
            message.delete()
        return True
    except Exception as e:
        print(e)
        return False

In [ ]:
# create_sqs_queue('main-protected-queue.fifo')
# send_message_to_sqs_queue('main-protected-queue.fifo', 'Hello World!', '1', '1')
# get_all_messages_from_sqs_queue('main-protected-queue.fifo')
# delete_all_messages_from_sqs_queue('main-protected-queue.fifo')

### S3 - APP INTERFACE

In [ ]:
# CREATE S3 BUCKET:
def create_s3_bucket(bucket_name):
    try:
        s3.create_bucket(
            Bucket=bucket_name,
            ObjectOwnership='BucketOwnerPreferred'
        )
    except Exception as e:
        print(e)
    return s3.Bucket(bucket_name)

# DELETE S3 BUCKET:
def delete_s3_bucket(bucket_name):
    try:
        bucket = s3.Bucket(bucket_name)
        bucket.objects.all().delete()
        bucket.delete()
    except Exception as e:
        print(e)

# PRINT S3 BUCKET DETAILS:
def print_s3_bucket_details(bucket):
    print("{name=%s, creation_date=%s}" % (bucket.name, bucket.creation_date))

# VIEW ALL S3 BUCKETS:
def view_all_s3_buckets():
    for bucket in s3.buckets.all():
        print_s3_bucket_details(bucket)

# UPLOAD LOCAL FILE TO S3 BUCKET:
def upload_local_file_to_s3(filename, bucketname, destination_path):
    s3.Bucket(bucketname).upload_file(filename, destination_path+filename)

# DELETE FILE FROM S3 BUCKET:
def delete_file_from_s3(filename, bucketname, path):
    s3.Bucket(bucketname).Object(path+filename).delete()

# DELETE DIRECTORY FROM S3 BUCKET:
def delete_directory_from_s3_bucket(bucket_name, directory_name):
    s3 = boto3.resource('s3')
    bucket = s3.Bucket(bucket_name)
    bucket.objects.filter(Prefix=directory_name).delete()

### EC2 - APP INTERFACE

In [ ]:
# START ALL EC2 INSTANCES:
def start_all_instances():
    # Start all instances:
    for instance in ec2.instances.filter(Filters=[{'Name': 'instance-state-name', 'Values': ['stopped']}]):
        instance.start()
        print('Starting instance: ', instance.id, instance.tags[0]['Value'])

# STOP ALL EC2 INSTANCES:
def stop_all_instances():
    # Stop all instances:
    for instance in ec2.instances.filter(Filters=[{'Name': 'instance-state-name', 'Values': ['running']}]):
        instance.stop()
        print('Stopping instance: ', instance.id, instance.tags[0]['Value'])

# PRINT EC2 INSTANCE DETAILS:
def print_instance_details(instance):
    print("{id=%s, name=%s, state=%s, type=%s}" % (instance.id, instance.tags[0]['Value'], instance.state['Name'], instance.instance_type))

# VIEW ALL EC2 INSTANCES:
def view_all_instances(include_terminated):
    # Print instance ID, name, state, and type:
    for instance in ec2.instances.all():
        if include_terminated or instance.state['Name'] != 'terminated':
            print_instance_details(instance)

# VIEW ALL EC2 INSTANCES - BY STATE FILTER:
def view_instances_by_state(state):
    # Print instance ID, name, state, and type; who are running
    for instance in ec2.instances.filter(Filters=[{'Name': 'instance-state-name', 'Values': [state]}]):
        print_instance_details(instance)

# TERMINATE ALL EC2 INSTANCES:
def terminate_all_instances():
    # Terminate all instances:
    for instance in ec2.instances.all():
        if instance.state['Name'] != 'terminated':
            instance.terminate()
            print('Terminated instance: ', instance.id, instance.tags[0]['Value'])

# TERMINATE EC2 INSTANCE - BY NAME:
def terminate_instance_by_name(name):
    for instance in ec2.instances.all():
        if instance.tags[0]['Value'] == name:
            instance.terminate()
            print('Terminated instance: ', instance.id, instance.tags[0]['Value'])

# CREATE INSTANCE - BY NAME, USING "amazon linux 2" AMI, "t2.micro" INSTANCE TYPE, "vockey" KEY PAIR, AND "default" SECURITY GROUP:
def create_instance(name):
    ec2.create_instances(
        ImageId='ami-0b0dcb5067f052a63',
        MinCount=1,
        MaxCount=1,
        InstanceType='t2.micro',
        KeyName='vockey',
        SecurityGroupIds=['sg-0f52fa9fe5477133b'],
        TagSpecifications=[
            {
                'ResourceType': 'instance',
                'Tags': [
                    {
                        'Key': 'Name',
                        'Value': name
                    },
                ]
            },
        ]
    )

# UPDATE INSTANCE AWS CREDENTIALS:
def update_instance_credentials_using_boto3_session_credentials(instance_name):
    exec_SSH_on_instance(instance_name, 'aws configure set aws_access_key_id '+get_boto3_session_credentials().access_key)
    exec_SSH_on_instance(instance_name, 'aws configure set aws_secret_access_key '+get_boto3_session_credentials().secret_key)
    exec_SSH_on_instance(instance_name, 'aws configure set aws_session_token '+get_boto3_session_credentials().token)
    exec_SSH_on_instance(instance_name, 'aws configure set region us-east-1')

# GET INSTANCE ID - BY NAME:
def get_instance_id_by_name(name):
    for instance in ec2.instances.all():
        if instance.tags[0]['Value'] == name and instance.state['Name'] != 'terminated':
            return instance.id

# GET INSTANCE PUBLIC IP - BY NAME:
def get_instance_public_dns_by_name(name):
    for instance in ec2.instances.all():
        if instance.tags[0]['Value'] == name:
            return instance.public_dns_name

# GET INSTANCE PUBLIC IP - BY ID:
def get_instance_public_dns_by_id(instance_id):
    for instance in ec2.instances.all():
        if instance.id == instance_id:
            return instance.public_dns_name

# EXECUTE SSH COMMAND ON INSTANCE - BY NAME:
def exec_SSH_on_instance(instance_name, command):
    paramiko_key = paramiko.RSAKey.from_private_key_file(pem_key)
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    try:
        client.connect(hostname=get_instance_public_dns_by_id(get_instance_id_by_name(instance_name)), username='ec2-user', pkey=paramiko_key)
        stdin, stdout, stderr = client.exec_command(command)
        return stdout.read(), stderr.read()
    except Exception as e:
        return e

### MATRIX - FUNCTIONS

In [ ]:
# Create a matrix of nxn size:
def create_random_square_matrix(n):
    return np.random.randint(0, 10, size=(n, n))

# Split matrix intro n row chunks:
def split_matrix_in_blocks(matrix, amount_of_blocks):
    factor = np.sqrt(amount_of_blocks)
    if not np.sqrt(amount_of_blocks).is_integer():
        raise ValueError('ERROR: Amount of blocks must be a square number!')
    elif matrix.shape[0] % factor != 0:
        raise ValueError('ERROR: Matrix size must be a multiple of factor!')
    else:
        sub_matrices = np.empty((int(factor), int(factor)), dtype=np.ndarray)
        pad = matrix.shape[0] / factor
        for i in range(0, sub_matrices.shape[0]):
            for j in range(0, sub_matrices.shape[0]):
                sub_matrices[i][j] = matrix[int(i * pad):int((i + 1) * pad), int(j * pad):int((j + 1) * pad)]
    return sub_matrices

def compute_single_block(A, B, i, j, size):
    block = np.dot(
        np.concatenate([A[i][k] for k in range(size)], axis=1),
        np.concatenate([B[k][j] for k in range(size)], axis=0)
    )
    return block

# Multiply two matrices:
def multiply_matrices(matrix1, matrix2, amount_of_blocks):
    # Split matrices into blocks:
    A = split_matrix_in_blocks(matrix1, amount_of_blocks)
    B = split_matrix_in_blocks(matrix2, amount_of_blocks)
    result = np.empty((A.shape[0], B.shape[1]), dtype=np.ndarray)
    size = A.shape[0]
    for i in range(0, result.shape[0]):
        for j in range(0, result.shape[1]):
            result[i][j] = compute_single_block(A, B, i, j, size)
    result = np.concatenate([np.concatenate([result[i][j] for j in range(size)], axis=1) for i in range(size)], axis=0)
    return result

## AWS - SOLUTION SETUP AND TASKS EXECUTION:

In [ ]:
# start_all_instances()

In [ ]:
view_all_instances(False)
instances = [instance for instance in ec2.instances.all() if instance.state['Name'] != 'terminated']
instances_names = [instance.tags[0]['Value'] for instance in instances]

In [ ]:
# Update all instances credentials:
for name in instances_names:
    update_instance_credentials_using_boto3_session_credentials(name)

In [ ]:
# Get all instances public DNS:
for name in instances_names:
    print(name+'\'s public DNS: '+get_instance_public_dns_by_name(name))

In [ ]:
# stop_all_instances()